In [2]:
# ✅ Fine-tuning MarianMT (Helsinki-NLP/opus-mt-uk-en) на базі HPLT у Google Colab або локальному середовищі
from transformers import MarianTokenizer, MarianMTModel, Seq2SeqTrainingArguments, Seq2SeqTrainer, DataCollatorForSeq2Seq
from datasets import Dataset
import pandas as pd
import torch
import os

# 🔘 1. Визначення режиму: Google Colab чи локальне середовище
USE_COLAB = "COLAB_GPU" in os.environ

if USE_COLAB:
    from google.colab import drive
    import os
    print("📂 Підключення Google Drive...")
    drive.mount('/content/drive')

file_path = "/content/drive/MyDrive/train/uk-en_dataset.tsv"

# Параметри
target_sample_size = 100000
chunk_size = 50000  # читаємо по 50 тис. рядків

samples = []
total_collected = 0

# Читаємо по частинах
for chunk in pd.read_csv(file_path, sep="\t", names=["uk", "en"], quoting=3, chunksize=chunk_size):
    # Фільтруємо непорожні, достатньої довжини
    chunk = chunk.dropna().query("uk.str.len() > 3 and en.str.len() > 3", engine="python")

    # Вибираємо частину з chunk
    available = len(chunk)
    need = target_sample_size - total_collected
    if available > need:
        chunk = chunk.sample(n=need, random_state=42)

    samples.append(chunk)
    total_collected += len(chunk)

    if total_collected >= target_sample_size:
        break

# Обʼєднуємо все в один датафрейм
df = pd.concat(samples).reset_index(drop=True)

print(f"✅ Отримано {len(df)} пар для тренування.")

# 🟡 Використовуємо  100 тисяч прикладів
#df = df.sample(n=100000, random_state=42)

# 📊 3. Конвертація в HuggingFace Dataset
dataset = Dataset.from_pandas(df)
dataset = dataset.train_test_split(test_size=0.05)


# 🔡 4. Завантаження токенізатора та моделі
model_name = "Helsinki-NLP/opus-mt-uk-en"
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)

# 🔧 5. Препроцесинг
def preprocess(example):
    inputs = tokenizer(example["uk"], max_length=128, padding="max_length", truncation=True)
    targets = tokenizer(example["en"], max_length=128, padding="max_length", truncation=True)
    inputs["labels"] = targets["input_ids"]
    return inputs

tokenized_dataset = dataset.map(preprocess, batched=True, remove_columns=dataset["train"].column_names)

# ⚙️ 6. Аргументи тренування
training_args = Seq2SeqTrainingArguments(
    output_dir="./marianmt-uk-en-hplt",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    logging_dir="./logs",
    #evaluation_strategy="epoch",
    save_strategy="epoch",
    predict_with_generate=True,
    fp16=torch.cuda.is_available(),
    save_total_limit=2,
)

# ▶️ 7. Створення Trainer
!pip install wandb
import os
os.environ["WANDB_DISABLED"] = "true"

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=DataCollatorForSeq2Seq(tokenizer, model=model)
)


# 🚀 8. Старт тренування
trainer.train()

# 💾 9. Збереження моделі
output_path = "./marianmt-uk-en-hplt-final"
trainer.save_model(output_path)


!zip -r marianmt-uk-en-hplt-final.zip marianmt-uk-en-hplt-final

from google.colab import files
files.download("marianmt-uk-en-hplt-final.zip")

📂 Підключення Google Drive...
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Отримано 100000 пар для тренування.


/usr/local/lib/python3.11/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


Map:   0%|          | 0/95000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


...

Step,Training Loss
500,1.440200
1000,1.024600
1500,0.909700
2000,0.856500
2500,0.817100
3000,0.782100
3500,0.749900
4000,0.727700
4500,0.712800
5000,0.701400


/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:3339: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 512, 'num_beams': 6, 'bad_words_ids': [[61586]]}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


  adding: marianmt-uk-en-hplt-final/ (stored 0%)
  adding: marianmt-uk-en-hplt-final/model.safetensors (deflated 7%)
  adding: marianmt-uk-en-hplt-final/generation_config.json (deflated 43%)
  adding: marianmt-uk-en-hplt-final/vocab.json (deflated 78%)
  adding: marianmt-uk-en-hplt-final/source.spm (deflated 57%)
  adding: marianmt-uk-en-hplt-final/tokenizer_config.json (deflated 68%)
  adding: marianmt-uk-en-hplt-final/config.json (deflated 62%)
  adding: marianmt-uk-en-hplt-final/target.spm (deflated 51%)
  adding: marianmt-uk-en-hplt-final/special_tokens_map.json (deflated 35%)
  adding: marianmt-uk-en-hplt-final/training_args.bin (deflated 52%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [3]:
!pip install transformers evaluate sacrebleu
# 📊 10. Оцінка моделі за допомогою sacreBLEU
from datasets import load_metric

# Завантаження BLEU метрики
bleu = load_metric("sacrebleu")  # або "bleu"

# Отримання результатів моделі на тестовому датасеті
raw_preds = trainer.predict(tokenized_dataset["test"])

# Декодування
preds = tokenizer.batch_decode(raw_preds.predictions, skip_special_tokens=True)
labels = tokenizer.batch_decode(raw_preds.label_ids, skip_special_tokens=True)

# sacrebleu очікує: list of predictions, list of list of references
references = [[ref] for ref in labels]

# Розрахунок BLEU
bleu_score = bleu.compute(predictions=preds, references=references)
print(f"\n🎯 sacreBLEU: {bleu_score['score']:.2f}")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 10.0 MB/s eta 0:00:00


<ipython-input-3-33ef1bdd2bf6>:6: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  bleu = load_metric("sacrebleu")  # або "bleu"



🎯 sacreBLEU: 27.42
